<a href="https://colab.research.google.com/github/AaryanChethan/aaryan_mit-/blob/main/Fakenewsprog1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
%pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
#gauth.LocalWebserverAuth()  # Creates a local webserver for authentication
drive = GoogleDrive(gauth)

file_id = 'YOUR_FILE_ID'  # Replace with your file ID
downloaded = drive.CreateFile({'id': file_id})
df = downloaded.GetContentFile("merged_df.xlsx")  # Save to local


# Load the dataset
df = pd.read_excel("D:\Fake News Detection\merged_df.xlsx")
with open(merged_df.xlsx, 'r') as fp:
    obj = json.load(fp)

# Encode labels: 0 for FAKE, 1 for REAL
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Step 1: Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Step 2: Define a custom dataset class for tokenizing input text
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize and encode the input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create Dataset objects for training and testing
MAX_LEN = 256  # Maximum length for padding/truncation
train_dataset = NewsDataset(X_train.tolist(), y_train.tolist(), tokenizer, MAX_LEN)
test_dataset = NewsDataset(X_test.tolist(), y_test.tolist(), tokenizer, MAX_LEN)

# Step 3: Load the pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)

# Step 4: Define training arguments and initialize the Trainer
training_args = TrainingArguments(
    output_dir='./results',              # Output directory for model and logs
    num_train_epochs=2,                  # Number of training epochs
    per_device_train_batch_size=8,       # Batch size for training (smaller batch size for mini-batch training)
    per_device_eval_batch_size=8,        # Batch size for evaluation
    evaluation_strategy='epoch',         # Evaluate after each epoch
    save_strategy='epoch',               # Save model checkpoints after each epoch
    logging_dir='./logs',                # Log directory
    logging_steps=100,
    gradient_accumulation_steps=4,       # Accumulate gradients across 4 steps to simulate larger batches
    fp16=True,                           # Enable mixed precision training for faster training
    load_best_model_at_end=True,         # Load the best model at the end of training
)

# Custom function for computing metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc}

# Step 5: Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Step 6: Train the model
trainer.train()

# Step 7: Evaluate the model on the test data
trainer.evaluate()

# Step 8: Get classification report
predictions = trainer.predict(test_dataset).predictions
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, predicted_labels))
print("\nClassification Report:\n", classification_report(y_test, predicted_labels))

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
# Provide path to your client_secrets.json file, or place it in the same directory as your script
gauth.LoadClientConfigFile("D:\Fake News Detection\merged_df.xlsx/client_secrets.json")
# If client_secrets.json is in the same directory, you can comment out the line above and uncomment the line below
# gauth.LocalWebserverAuth()  # This will open a browser window for authentication

drive = GoogleDrive(gauth)

file_id = '114025291535706042385'  # Replace with your file ID
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("merged_df.xlsx")
# Authenticate and download the dataset from Google Drive


# Load the dataset
df = pd.read_excel("merged_df.xlsx")

# Encode labels: 0 for FAKE, 1 for REAL
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Step 1: Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Step 2: Define a custom dataset class for tokenizing input text
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize and encode the input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create Dataset objects for training and testing
MAX_LEN = 128  # Adjusting sequence length based on your dataset size
train_dataset = NewsDataset(X_train.tolist(), y_train.tolist(), tokenizer, MAX_LEN)
test_dataset = NewsDataset(X_test.tolist(), y_test.tolist(), tokenizer, MAX_LEN)

# Step 3: Load the pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)

# Step 4: Define training arguments and initialize the Trainer
training_args = TrainingArguments(
    output_dir='./results',              # Output directory for model and logs
    num_train_epochs=2,                  # Number of training epochs
    per_device_train_batch_size=8,       # Batch size for training (smaller batch size for mini-batch training)
    per_device_eval_batch_size=8,        # Batch size for evaluation
    evaluation_strategy='epoch',         # Evaluate after each epoch
    save_strategy='epoch',               # Save model checkpoints after each epoch
    logging_dir='./logs',                # Log directory
    logging_steps=100,
    gradient_accumulation_steps=4,       # Accumulate gradients across 4 steps
    fp16=True,                           # Enable mixed precision training (disable if necessary)
    load_best_model_at_end=True,         # Load the best model at the end of training
)

# Custom function for computing metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc}

# Step 5: Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Step 6: Train the model
trainer.train()

# Step 7: Evaluate the model on the test data
trainer.evaluate()

# Step 8: Get classification report
predictions = trainer.predict(test_dataset).predictions
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, predicted_labels))
print("\nClassification Report:\n", classification_report(y_test, predicted_labels))


InvalidConfigError: Invalid client secrets file ('Error opening file', 'D:\\Fake News Detection\\merged_df.xlsx/client_secrets.json', 'No such file or directory', 2)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving merged_df.xlsx to merged_df (1).xlsx


In [ ]:
!pwd


/content


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_excel("merged_df.xlsx")

# Encode labels: 0 for FAKE, 1 for REAL
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Step 1: Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Step 2: Define a custom dataset class for tokenizing input text
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize and encode the input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create Dataset objects for training and testing
MAX_LEN = 256  # Maximum length for padding/truncation
train_dataset = NewsDataset(X_train.tolist(), y_train.tolist(), tokenizer, MAX_LEN)
test_dataset = NewsDataset(X_test.tolist(), y_test.tolist(), tokenizer, MAX_LEN)

# Step 3: Load the pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)

# Step 4: Define training arguments and initialize the Trainer
training_args = TrainingArguments(
    output_dir='./results',              # Output directory for model and logs
    num_train_epochs=2,                  # Number of training epochs
    per_device_train_batch_size=8,       # Batch size for training (smaller batch size for mini-batch training)
    per_device_eval_batch_size=8,        # Batch size for evaluation
    evaluation_strategy='epoch',         # Evaluate after each epoch
    save_strategy='epoch',               # Save model checkpoints after each epoch
    logging_dir='./logs',                # Log directory
    logging_steps=100,
    gradient_accumulation_steps=4,       # Accumulate gradients across 4 steps to simulate larger batches
    fp16=True,                           # Enable mixed precision training for faster training
    load_best_model_at_end=True,         # Load the best model at the end of training
)

# Custom function for computing metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits from numpy array to tensor
    logits = torch.tensor(logits)
    predictions = torch.argmax(logits, dim=-1)
    acc = accuracy_score(labels, predictions.numpy())  # Convert labels to numpy for accuracy calculation
    return {'accuracy': acc}


# Step 5: Initialize the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Step 6: Train the model
trainer.train()

# Step 7: Evaluate the model on the test data
trainer.evaluate()

# Step 8: Get classification report
predictions = trainer.predict(test_dataset).predictions
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, predicted_labels))
print("\nClassification Report:\n", classification_report(y_test, predicted_labels))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/p

Epoch,Training Loss,Validation Loss,Accuracy
0,0.149300,0.144072,0.935766
1,0.113800,0.146287,0.937636


Accuracy: 0.9357660958648022

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.89      0.91      5140
           1       0.94      0.96      0.95      8233

    accuracy                           0.94     13373
   macro avg       0.94      0.93      0.93     13373
weighted avg       0.94      0.94      0.94     13373



# New Section